# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import os
import geopandas as gpd
from Bio import Entrez
import time
from tqdm import tqdm
import requests
from Bio import Entrez
import gzip
import subprocess 
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Read the data

In [ ]:
# Read the data 
df = pd.read_csv("/home/chandru/binp37/results/metasub/processed_metasub.csv")
df.head()

In [ ]:
rfe_df = pd.read_csv("/home/chandru/binp37/results/metasub/metasub_training_testing_data.csv")
rfe_df.head()

# Geograpical features

In [ ]:
feature_data = df[['city_total_population','city_population_density',
                  'city_land_area_km2','city_ave_june_temp_c','city_elevation_meters','city_koppen_climate','continent','city','latitude','longitude']]

# Fix city elevation of hanoi, yamaguchi in meters
feature_data.loc[feature_data['city'] == 'hanoi','city_elevation_meters'] = 12
feature_data.loc[feature_data['city'] == 'yamaguchi','city_elevation_meters'] = 23
feature_data.loc[feature_data['city'] == 'marseille','city_elevation_meters'] = 42 # city elevation of marseille on google is 42 m here it is 0

# Get city population density, city ladn ares in km2, city avg temp in june and city elevation in meters of offa 
offa_data = {
    'city_population_density': 2500.0,
    'city_land_area_km2': 74.0,
    'city_ave_june_temp_c': 28.0,
    'city_elevation_meters': 457.0
}

feature_data.loc[feature_data['city'] == 'offa', list(offa_data.keys())] = list(offa_data.values())

# Get city land area in km2 of marseille  
feature_data.loc[feature_data['city'] == 'marseille','city_land_area_km2'] = 240

# Fix all the nan values of london
london_data = {
    'city_total_population': 8787892.0,
    'city_population_density': 5590.0,
    'city_land_area_km2': 1572.0,
    'city_ave_june_temp_c': 14.4,
    'city_elevation_meters': 11.0,
    'city_koppen_climate': 'marine_west_coast_climate'
}
feature_data.loc[feature_data['city'] == 'london', list(london_data.keys())] = list(london_data.values())


feature_data.head()

## Scaling the features

In [ ]:
# Check for skewness in the data before appling long transformer -> 
# Note to self: The city_land_area_km2 is right skewed, so we will go with log scale transformation
#             : The city_elevation_meters is multi modal there we will go with QuantileTransformer


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, QuantileTransformer, OneHotEncoder
import numpy as np
import pandas as pd

# Step 1: Define input columns
#log_cols = ['city_land_area_km2']
#quantile_cols = ['city_elevation_meters']
scale_cols = ['city_total_population', 'city_ave_june_temp_c']
cat_cols = ['city_koppen_climate']

# Step 2: Log-transform function
#def safe_log1p(x):
#    return np.log1p(np.maximum(x, 0))

# Step 3: Create log pipeline
#log_pipeline = Pipeline([
#    ('log', FunctionTransformer(safe_log1p)),
#    ('scale', StandardScaler())
#])

# Step 4: Build the ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
#    ('log', log_pipeline, log_cols),
#    ('quantile', QuantileTransformer(output_distribution='normal'), quantile_cols),
    ('scale', StandardScaler(), scale_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

# Step 5: Fit and transform
geo_features_processed = preprocessor.fit_transform(feature_data)

# Step 6: Extract column names correctly
output_feature_names = []

for name, transformer, cols in preprocessor.transformers_:
    if name == 'cat':
        # For OneHotEncoder
        encoder = transformer
        if isinstance(encoder, Pipeline):
            encoder = encoder.named_steps['onehot']
        cats = encoder.categories_[0]
        output_feature_names.extend([f"{cols[0]}_{cat}" for cat in cats])
    else:
        output_feature_names.extend(cols)

# Step 7: Convert to DataFrame
geo_features_df = pd.DataFrame(geo_features_processed)

# Step 8: Merge with main features (RFE-selected ones)
final_df = pd.concat([rfe_df, geo_features_df], axis=1)
final_df.to_csv("/home/chandru/binp37/results/metasub/metasub_geo_training_testing.csv", index=False)

print("Final dataset shape:", final_df.shape)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Step 1: Select your input columns
scale_cols = ['city_ave_june_temp_c']
#cat_cols = ['city_koppen_climate']

# Step 2: Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale_cols),
#        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

# Step 3: Fit and transform the geo feature data
geo_features_processed = preprocessor.fit_transform(feature_data)

# Step 4: Get feature names
feature_names = []

# Handle scaled columns
feature_names.extend(scale_cols)

# Handle one-hot columns
#ohe = preprocessor.named_transformers_['cat']
#cat_feature_names = ohe.get_feature_names_out(cat_cols)
#feature_names.extend(cat_feature_names)

# Step 5: Convert to DataFrame
geo_features_df = pd.DataFrame(geo_features_processed.toarray() if hasattr(geo_features_processed, 'toarray') else geo_features_processed)

# Step 6: Merge with selected features and save
final_df = pd.concat([rfe_df.reset_index(drop=True), geo_features_df.reset_index(drop=True)], axis=1)
#final_df.to_csv("/home/chandru/binp37/results/metasub/metasub_geo_training_testing.csv", index=False)

print("Final dataset shape:", final_df.shape)


In [ ]:
df = pd.read_csv("/home/chandru/binp37/results/metasub/metasub_geo_training_testing.csv")
df.head()

In [ ]:
df.columns

# Microbiome features

In [ ]:
# We can get the raw sequence of all these top hundered species and get a phylogenetic tree to determine the relationship between species.
# We can then use the information as well as a feature to predict the lat and long.

microbe_data = rfe_df.iloc[:,:-4]
microbe_data

## Phylogenetic Trees

In [ ]:
species_list = []
for name in microbe_data.columns:
    species_list.append(name)
    

tax_df = pd.read_csv("/home/chandru/binp37/results/metasub/taxonomic_info.csv")
lin_df = tax_df[tax_df['Species'].isin(species_list)].dropna(axis=1,how='all')
lin_df = lin_df.dropna(subset=lin_df.columns[1:7]).iloc[:,:7]
lin_df.head()

In [ ]:
print(np.unique(lin_df['Rank_1'],return_counts=True)[0],np.unique(lin_df['Rank_1'],return_counts=True)[1])

In [ ]:
# Count values
counts = lin_df['Rank_2'].value_counts()

# Plot
plt.figure(figsize=(6, 4))
counts.plot(kind='bar', color=['tomato', 'skyblue'])
plt.title('Frequency of Rank_1 Categories')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

Entrez.email = "1ms19bt011@gmail.com" # Remember to set your actual email

def download_genome(species, output_dir="genomes"):
    """
    Downloads the complete genome for a given species from NCBI RefSeq,
    handling both FTP and HTTP URLs.

    Args:
        species (str): The scientific name of the species (e.g., "Escherichia coli").
        output_dir (str): The directory where the genome file will be saved.

    Returns:
        bool: True if the genome was successfully downloaded and decompressed, False otherwise.
    """
    os.makedirs(output_dir, exist_ok=True)

    search_terms = [
        f'"{species}"[Organism] AND "complete genome"[Assembly Level]',
        f'"{species}"[Organism] AND "reference genome"[Refseq Category]',
        f'"{species}"[Organism] AND latest[filter]',
        f'"{species}"[Organism]' # Broadest term as a last resort
    ]

    for term_index, term in enumerate(search_terms):
        print(f"Searching for '{species}' with term: '{term}'")
        try:
            # Search for latest RefSeq assembly
            handle = Entrez.esearch(db="assembly", term=term, retmax=1)
            record = Entrez.read(handle)
            handle.close() # Always close the handle

            if record["IdList"]:
                assembly_id = record["IdList"][0]
                print(f"Found assembly ID: {assembly_id} for {species}")

                # Fetch summary to get FTP path
                summary_handle = Entrez.esummary(db="assembly", id=assembly_id)
                doc = Entrez.read(summary_handle)
                summary_handle.close() # Always close the handle

                ftp_path = doc["DocumentSummarySet"]["DocumentSummary"][0]["FtpPath_RefSeq"]
                if ftp_path:
                    filename_stem = ftp_path.split("/")[-1]
                    fasta_url = f"{ftp_path}/{filename_stem}_genomic.fna.gz"
                    output_gz_path = os.path.join(output_dir, f"{species.replace(' ', '_')}.fna.gz")
                    output_fna_path = os.path.join(output_dir, f"{species.replace(' ', '_')}.fna")

                    print(f"Attempting to download from: {fasta_url}")

                    try:
                        if fasta_url.startswith("ftp://"):
                            # Use wget for FTP paths
                            print(f"Using wget for FTP download: {fasta_url}")
                            # -q for quiet, -O for output file, --show-progress for progress bar
                            # --no-verbose for cleaner output
                            # Use subprocess.run for better control and error handling than os.system
                            result = subprocess.run(
                                ["wget", "--no-verbose", "--show-progress", "-O", output_gz_path, fasta_url],
                                check=True, # Raise CalledProcessError if wget returns non-zero exit code
                                capture_output=True, # Capture stdout/stderr for debugging if needed
                                text=True # Decode stdout/stderr as text
                            )
                            # print(result.stdout) # Uncomment for detailed wget output
                            # print(result.stderr) # Uncomment for detailed wget output
                            print(f"Downloaded {species} to {output_gz_path} using wget.")
                        else:
                            # Use requests for HTTP/HTTPS paths
                            print(f"Using requests for HTTP/HTTPS download: {fasta_url}")
                            response = requests.get(fasta_url, stream=True)
                            response.raise_for_status() # Raise an exception for HTTP errors

                            total_size_in_bytes = int(response.headers.get('content-length', 0))
                            block_size = 1024 # 1 Kibibyte
                            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc=f"Downloading {species}")

                            with open(output_gz_path, 'wb') as f:
                                for chunk in response.iter_content(chunk_size=block_size):
                                    progress_bar.update(len(chunk))
                                    f.write(chunk)
                            progress_bar.close()

                            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                                print("ERROR, something went wrong during download!")
                                return False
                            print(f"Downloaded {species} to {output_gz_path} using requests.")

                        # Decompress the file, regardless of how it was downloaded
                        print(f"Decompressing {output_gz_path}...")
                        with gzip.open(output_gz_path, 'rb') as f_in:
                            with open(output_fna_path, 'wb') as f_out:
                                f_out.write(f_in.read())
                        os.remove(output_gz_path) # Remove the compressed file
                        print(f"Decompressed to {output_fna_path}")
                        return True
                    except subprocess.CalledProcessError as sub_e:
                        print(f"wget failed for {species} from {fasta_url}: {sub_e}")
                        print(f"wget stdout: {sub_e.stdout}")
                        print(f"wget stderr: {sub_e.stderr}")
                        continue # Try next search term
                    except requests.exceptions.RequestException as req_e:
                        print(f"Download failed for {species} from {fasta_url}: {req_e}")
                        continue # Try next search term
                    except Exception as download_e:
                        print(f"An unexpected error occurred during download/decompression for {species}: {download_e}")
                        continue # Try next search term
                else:
                    print(f"No FTP path found for {species} with term '{term}'. Trying next search term.")
            else:
                print(f"No assembly found for {species} with term '{term}'. Trying next search term.")
            time.sleep(1) # Small delay between Entrez calls to be polite
        except Exception as e:
            print(f"Error during Entrez search for {species} with term '{term}': {e}")
            time.sleep(2) # Longer delay if Entrez call itself fails
    print(f"Failed to download genome for {species} after trying all search terms.")
    return False


output_directory = "genomes"
os.makedirs(output_directory, exist_ok=True)

# Process each species in the list
print("\nStarting genome download process...")
for species in tqdm(filtered_species_list[:], desc="Overall Genome Download Progress"):
    print(f"\nProcessing species: {species}")
    success = download_genome(species, output_directory)
    if not success:
        print(f"Could not download genome for {species}. Please check the species name or try again later.")
    time.sleep(2) # Respect NCBI rate limits between species

## Clutering using K-means

In [ ]:
# Elbow method to determine ideal cluster size -> Note I am getting the cut iff to be 15.
# Calculate inertia for k=1 to 50
inertias = []
for k in range(1, 50):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(np.array(microbe_data))
    inertias.append(kmeans.inertia_)

# Plot Elbow Curve
plt.plot(range(1, 50), inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=15, random_state=42, n_init="auto").fit(np.array(microbe_data))
kmeans.cluster_centers_


centorid_distances = cdist(np.array(microbe_data),kmeans.cluster_centers_,"euclidean")
closet_indices = np.argmin(centorid_distances,axis=0)

augment_data = pd.concat([microbe_data,pd.DataFrame(centorid_distances)],axis=1)
augment_data

# Error Analysis

In [2]:
# Data processing function for hierarchical model
def process_data_hierarchical(df):
    """Process data for hierarchical prediction"""
    # Process continuous features
    cont_cols = [col for col in df.columns if col not in [
        'latitude', 'longitude',
        'latitude_rad', 'longitude_rad', 'x', 'y', 'z',
        'scaled_x', 'scaled_y', 'scaled_z', 'continent', 'city'
    ]]
    
    # Get the features
    x_cont = df[cont_cols].values
    
    # Encode continent labels
    continent_encoder = LabelEncoder()
    y_continent = continent_encoder.fit_transform(df['continent'].values)
    
    # Encode city labels
    city_encoder = LabelEncoder()
    y_city = city_encoder.fit_transform(df['city'].values)
    
    # Calculate coordinates if not already present
    if not all(col in df.columns for col in ['x', 'y', 'z']):
        df['latitude_rad'] = np.deg2rad(df['latitude'])
        df['longitude_rad'] = np.deg2rad(df['longitude'])
        df['x'] = np.cos(df['latitude_rad']) * np.cos(df['longitude_rad'])
        df['y'] = np.cos(df['latitude_rad']) * np.sin(df['longitude_rad'])
        df['z'] = np.sin(df['latitude_rad'])
    
    # Scale coordinates
    coord_scaler = StandardScaler()
    y_coords = coord_scaler.fit_transform(df[['x', 'y', 'z']].values)
    
    continents = continent_encoder.classes_
    cities = city_encoder.classes_
    
    print(f"Continents: {len(continents)} ({continents})")
    print(f"Cities: {len(cities)}")
    print(f"Continuous features: {len(cont_cols)}")
    
    return {
        'x_cont': x_cont,
        'y_continent': y_continent,
        'y_city': y_city,
        'y_coords': y_coords, # This is for neural networks. Scaling is required
        'y_latitude': df['latitude'].values, # This is for XGBoost, we don't need to scale this
        'y_longitude':df['longitude'].values, # This is for XGBoost, we don't need to scale this
        'encoders': {
            'continent': continent_encoder,
            'city': city_encoder,
            'coord': coord_scaler
        },
        'continents': continents,
        'cities': cities
    }

df = pd.read_csv("/home/chandru/binp37/results/metasub/metasub_training_testing_data.csv")
processed_data = process_data_hierarchical(df)

Continents: 7 (['east_asia' 'europe' 'middle_east' 'north_america' 'oceania'
 'south_america' 'sub_saharan_africa'])
Cities: 40
Continuous features: 200


In [3]:
processed_data['continents']

array(['east_asia', 'europe', 'middle_east', 'north_america', 'oceania',
       'south_america', 'sub_saharan_africa'], dtype=object)

In [4]:
os.chdir("/home/chandru/binp37/scripts/ensemble")
y_test_cont = np.load("saved_results/y_test_cont.npy")
y_pred_cont = np.load("saved_results/y_pred_cont.npy")

y_test_city = np.load("saved_results/y_test_city.npy")
y_pred_city = np.load("saved_results/y_pred_city.npy")

y_test_coords = np.load("saved_results/y_test_coord.npy")
y_pred_coords = np.load("saved_results/y_pred_coord.npy")


In [5]:
df = pd.DataFrame({
    'true_cont': y_test_cont,
    'pred_cont': y_pred_cont,
    'true_city': y_test_city,
    'pred_city': y_pred_city,
    'true_lat': y_test_coords[:, 0],
    'true_lon': y_test_coords[:, 1],
    'pred_lat': y_pred_coords[:, 0],
    'pred_lon': y_pred_coords[:, 1]
})

In [6]:
continents = np.array([
    'east_asia', 'europe', 'middle_east', 'north_america',
    'oceania', 'south_america', 'sub_saharan_africa'
])

cities = np.array([
    'auckland', 'baltimore', 'barcelona', 'berlin', 'bogota', 'brisbane',
    'denver', 'doha', 'europe', 'fairbanks', 'hamilton', 'hanoi',
    'hong_kong', 'ilorin', 'kuala_lumpur', 'kyiv', 'lisbon', 'london',
    'marseille', 'minneapolis', 'naples', 'new_york_city', 'offa', 'oslo',
    'paris', 'rio_de_janeiro', 'sacramento', 'san_francisco', 'santiago',
    'sao_paulo', 'sendai', 'seoul', 'singapore', 'sofia', 'stockholm',
    'taipei', 'tokyo', 'vienna', 'yamaguchi', 'zurich'
])


In [7]:
df['true_cont_name'] = df['true_cont'].map(lambda i: continents[i])
df['pred_cont_name'] = df['pred_cont'].map(lambda i: continents[i])

df['true_city_name'] = df['true_city'].map(lambda i: cities[i])
df['pred_city_name'] = df['pred_city'].map(lambda i: cities[i])


In [8]:
cont_support_map = dict(zip(np.unique(df['true_cont_name'],return_counts=True)[0],np.unique(df['true_cont_name'],return_counts=True)[1]))
city_support_map = dict(zip(np.unique(df['true_city_name'],return_counts=True)[0],np.unique(df['true_city_name'],return_counts=True)[1]))

In [9]:
# Step 1: Compute the correctness 
df['continent_correct'] = df['true_cont'] == df['pred_cont']
df['city_correct'] = df['true_city'] == df['pred_city']

In [10]:
# Step 2: Compute coordinates distance

# Distance between two points on the earth
def haversine_distance(lat1,lon1,lat2,lon2):
    """
    Calculate the great circle distance between two points on the earth
    """
    # Radius of the earth
    R = 6371.0

    # Convert from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2) **2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c # in kilometers



df['coord_error'] = haversine_distance(df['true_lat'].values,df['true_lon'].values,df['pred_lat'].values,df['pred_lon'].values)

In [30]:
print(f'The median distance error is {np.median(df['coord_error'].values)}')
print(f'The mean distance error is {np.mean(df['coord_error'].values)}')
print(f'The max distance error is {np.max(df['coord_error'].values)}')

The median distance error is 17.84439468383789
The mean distance error is 727.4476318359375
The max distance error is 15990.0634765625


In [32]:
# Sort the DataFrame by 'coord_error' in descending order
df_sorted = df.sort_values(by='coord_error', ascending=False)   
print("Top 10 largest coordinate errors:")
df_sorted[['true_city_name', 'pred_city_name', 'true_cont_name', 'pred_cont_name','true_lat','pred_lat','true_lon','pred_lon',
            'coord_error']].head(10)

Top 10 largest coordinate errors:


,true_city_name,pred_city_name,true_cont_name,pred_cont_name,true_lat,pred_lat,true_lon,pred_lon,coord_error
632,brisbane,sao_paulo,oceania,east_asia,-27.431326,48.476173,153.074997,11.324935,15990.063477
86,sao_paulo,sao_paulo,south_america,east_asia,-23.561726,16.010761,-46.656693,88.357361,15253.044922
208,singapore,kyiv,east_asia,east_asia,1.298330,-15.619979,103.798027,-33.606396,15081.753906
550,hamilton,auckland,oceania,oceania,-37.783329,40.442173,175.283325,-69.655579,14534.150391
354,london,london,europe,europe,51.480598,-15.746110,-0.194938,105.246605,12435.375977
70,fairbanks,hamilton,north_america,europe,64.837784,-38.060692,-147.716385,179.087601,11801.385742
254,tokyo,barcelona,east_asia,middle_east,35.605324,44.005825,139.682404,-25.795036,11041.908203
159,hanoi,ilorin,east_asia,europe,21.000000,9.228308,105.000000,6.301521,10529.858398
614,singapore,singapore,east_asia,europe,1.431875,16.023275,103.775139,12.307347,10120.409180
1,singapore,yamaguchi,east_asia,east_asia,1.326990,48.476173,103.946732,11.324935,10090.230469


In [44]:
# Get ROC AUC for continent and city - This a multiclass classification problem
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, auc
# Binarize the true labels for ROC AUC calculation
y_true_cont_binarized = label_binarize(y_test_cont, classes=np.arange(len(processed_data['continents'])))
y_true_city_binarized = label_binarize(y_test_city, classes=np.arange(len(processed_data['cities'])))

# Calculate ROC AUC for continent and city
roc_auc_continent = roc_auc_score(y_true_cont_binarized, label_binarize(y_pred_cont, classes=np.arange(len(processed_data['continents']))), average='macro', multi_class='ovr')
roc_auc_city = roc_auc_score(y_true_city_binarized, label_binarize(y_pred_city, classes=np.arange(len(processed_data['cities']))), average='macro', multi_class='ovr')

# Calculate Precision-Recall AUC for continent and city
precision_cont, recall_cont, _ = precision_recall_curve(y_true_cont_binarized.ravel(), label_binarize(y_pred_cont, classes=np.arange(len(processed_data['continents']))).ravel())
prc_auc_continent = auc(recall_cont, precision_cont)

precision_city, recall_city, _ = precision_recall_curve(y_true_city_binarized.ravel(), label_binarize(y_pred_city, classes=np.arange(len(processed_data['cities']))).ravel())
prc_auc_city = auc(recall_city, precision_city)

print(f"ROC AUC (Continent): {roc_auc_continent:.3f}")
print(f"ROC AUC (City): {roc_auc_city:.3f}")
print(f"PRC AUC (Continent): {prc_auc_continent:.3f}")
print(f"PRC AUC (City): {prc_auc_city:.3f}")


ROC AUC (Continent): 0.935
ROC AUC (City): 0.895
PRC AUC (Continent): 0.949
PRC AUC (City): 0.908


In [37]:
# Step 3: Group into the 4 categories
def group_label(row):
    if row['continent_correct'] and row['city_correct']:
        return 'C_correct Z_correct'
    elif row['continent_correct'] and not row['city_correct']:
        return 'C_correct Z_wrong'
    elif not row['continent_correct'] and row['city_correct']:
        return 'C_wrong Z_correct'
    else:
        return 'C_wrong Z_wrong'

df['error_group'] = df.apply(group_label, axis=1)

In [12]:
# Step 4: Aggregate stats
group_stats = df.groupby('error_group')['coord_error'].agg([
    ('count', 'count'),
    ('mean_error_km', 'mean'),
    ('median_error_km', 'median')
])

In [13]:
# Step 5: Calculate proportions and expected error
total = len(df)
group_stats['proportion'] = group_stats['count'] / total
group_stats['weighted_error'] = group_stats['mean_error_km'] * group_stats['proportion']
expected_total_error = group_stats['weighted_error'].sum()
print(f"\nExpected Coordinate Error E[D]: {expected_total_error:.2f} km")


Expected Coordinate Error E[D]: 727.45 km


In [45]:
def compute_in_radius_metrics(y_true, y_pred, thresholds=[1, 5, 50, 100, 250, 500, 1000, 5000]):
    """
    Compute % of predictions within given distance thresholds
    y_true, y_pred: numpy arrays of shape (N, 2) for [lat, lon]
    """
    distances = haversine_distance(
        y_true[:, 0], y_true[:, 1], y_pred[:, 0], y_pred[:, 1]
    )

    results = {}
    for r in thresholds:
        percent = np.mean(distances <= r) * 100
        results[f"<{r} km"] = percent
    
    return results, distances

metrics, dists = compute_in_radius_metrics(y_test_coords, y_pred_coords)

print("In-Radius Accuracy Metrics:")
for k, v in metrics.items():
    print(f"{k:>8}: {v:.2f}%")


In-Radius Accuracy Metrics:
   <1 km: 0.49%
   <5 km: 16.34%
  <50 km: 64.62%
 <100 km: 70.02%
 <250 km: 78.13%
 <500 km: 82.80%
<1000 km: 87.22%
<5000 km: 94.23%


In [15]:
def in_radius_by_group(df, group_col, thresholds=[1, 5, 50, 100, 250, 500, 1000, 5000]):
    """
    Compute in-radius accuracy for a group column (continent, city, or continent+city)
    """
    df = df.copy()
    df['coord_error'] = haversine_distance(
        df['true_lat'].values, df['true_lon'].values,
        df['pred_lat'].values, df['pred_lon'].values
    )

    results = {}
    grouped = df.groupby(group_col)
    
    for group_name, group_df in grouped:
        res = {}
        errors = group_df['coord_error'].values
        for r in thresholds:
            res[f"<{r} km"] = np.mean(errors <= r) * 100  # in %
        results[group_name] = res
    
    return pd.DataFrame(results).T  # Transpose for better readability


In [16]:
continent_metrics = in_radius_by_group(df, group_col='true_cont_name')
print("In-Radius Accuracy per Continent")
continent_metrics['continent_support'] = continent_metrics.index.map(cont_support_map)
continent_metrics.round(2)

In-Radius Accuracy per Continent


,<1 km,<5 km,<50 km,<100 km,<250 km,<500 km,<1000 km,<5000 km,continent_support
east_asia,0.72,25.90,67.63,71.58,80.58,84.89,88.85,94.96,278
europe,0.71,20.14,63.96,71.38,77.74,81.98,87.63,93.99,283
middle_east,0.00,0.00,80.00,86.67,86.67,93.33,93.33,100.00,15
north_america,0.00,2.68,56.38,61.74,71.14,78.52,83.89,93.96,149
oceania,0.00,0.00,0.00,0.00,22.22,22.22,22.22,44.44,9
south_america,0.00,0.00,47.62,52.38,71.43,80.95,85.71,90.48,21
sub_saharan_africa,0.00,0.00,86.44,89.83,94.92,94.92,94.92,100.00,59


In [17]:
city_metrics = in_radius_by_group(df, group_col='true_city_name')
print("In-Radius Accuracy per City")
city_metrics['city_support'] = city_metrics.index.map(city_support_map)
city_metrics.round(2)

In-Radius Accuracy per City


,<1 km,<5 km,<50 km,<100 km,<250 km,<500 km,<1000 km,<5000 km,city_support
auckland,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,1
baltimore,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,1
barcelona,0.00,0.00,100.00,100.00,100.00,100.00,100.00,100.00,23
berlin,0.00,0.00,6.67,40.00,86.67,93.33,100.00,100.00,15
bogota,0.00,0.00,0.00,0.00,25.00,50.00,50.00,75.00,4
brisbane,0.00,0.00,0.00,0.00,20.00,20.00,20.00,20.00,5
denver,0.00,0.00,60.00,73.33,86.67,86.67,93.33,100.00,15
doha,0.00,0.00,80.00,86.67,86.67,93.33,93.33,100.00,15
europe,0.00,33.33,83.33,91.67,100.00,100.00,100.00,100.00,12
fairbanks,0.00,14.29,38.10,42.86,57.14,61.90,61.90,76.19,21


In [18]:
df['continent_city'] = df['true_cont_name'] + " / " + df['true_city_name']
cont_city_metrics = in_radius_by_group(df, group_col='continent_city')
cont_city_metrics['continent_support'] = cont_city_metrics.index.map(lambda x :x.split("/")[-1].strip()).map(city_support_map)
print("In-Radius Accuracy per Continent-City")
cont_city_metrics.round(2)

In-Radius Accuracy per Continent-City


,<1 km,<5 km,<50 km,<100 km,<250 km,<500 km,<1000 km,<5000 km,continent_support
east_asia / hanoi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.00,5
east_asia / hong_kong,0.68,41.89,93.92,93.92,93.92,95.95,96.62,98.65,148
east_asia / kuala_lumpur,0.00,0.00,0.00,10.00,80.00,80.00,90.00,100.00,10
east_asia / sendai,0.00,0.00,0.00,0.00,50.00,100.00,100.00,100.00,4
east_asia / seoul,5.26,5.26,63.16,78.95,78.95,84.21,94.74,100.00,19
east_asia / singapore,0.00,0.00,0.00,9.38,31.25,43.75,53.12,78.12,32
east_asia / taipei,0.00,0.00,57.89,68.42,100.00,100.00,100.00,100.00,19
east_asia / tokyo,0.00,23.68,68.42,73.68,81.58,86.84,89.47,92.11,38
east_asia / yamaguchi,0.00,0.00,0.00,0.00,0.00,0.00,100.00,100.00,3
europe / barcelona,0.00,0.00,100.00,100.00,100.00,100.00,100.00,100.00,23
